In [2]:
# import require packages
import pandas as pd
import os
import argparse
import numpy as np
import statistics
import math
import re
import json
import requests

## As map macthing requires, OSM pbf file for a region. We choose Saxsony region

In [ ]:
import os
df_concat1 = pd.DataFrame(columns=["traj_id","latitude","longitude","city"])
directory = os.path.join("/home/dadwal/german_watermarkedData/watermarked_data/")
for root,dirs,files in os.walk(directory):
    for dir1 in dirs:
        if dir1!="watermarking":
            df_2 = pd.read_csv('/home/dadwal/german_watermarkedData/watermarked_data/'+str(dir1)+"/watermarking/watermarkedTraj.csv")
            df_2=df_2[["traj_id","latitude","longitude"]]
            df_2_first=df_2.head(1)
            df_2_first=df_2_first.apply(city_state_country, axis=1)
            df_concat1 = pd.concat([df_concat1, df_2_first])
            

In [38]:
df_concat1

,traj_id,latitude,longitude,city,state,country
0,758.0,50.8572,13.4729,Bobritzsch-Hilbersdorf,Saxony,Germany
0,421.0,51.1189,14.4454,,Saxony,Germany
0,270.0,50.8357,12.9280,Chemnitz,Saxony,Germany
0,975.0,51.0332,13.7856,Dresden,Saxony,Germany
0,670.0,50.6909,13.0306,,Saxony,Germany
...,...,...,...,...,...,...
0,100.0,51.0854,13.7372,,Saxony,Germany
0,216.0,51.0712,13.6445,,Saxony,Germany
0,518.0,51.0720,13.7909,Dresden,Saxony,Germany
0,4.0,50.9658,14.7018,,Saxony,Germany


In [41]:
df=df_concat1.loc[df_concat1["state"]=="Saxony"]

In [42]:
df

,traj_id,latitude,longitude,city,state,country
0,758.0,50.8572,13.4729,Bobritzsch-Hilbersdorf,Saxony,Germany
0,421.0,51.1189,14.4454,,Saxony,Germany
0,270.0,50.8357,12.9280,Chemnitz,Saxony,Germany
0,975.0,51.0332,13.7856,Dresden,Saxony,Germany
0,670.0,50.6909,13.0306,,Saxony,Germany
...,...,...,...,...,...,...
0,100.0,51.0854,13.7372,,Saxony,Germany
0,216.0,51.0712,13.6445,,Saxony,Germany
0,518.0,51.0720,13.7909,Dresden,Saxony,Germany
0,4.0,50.9658,14.7018,,Saxony,Germany


In [43]:
df.to_csv("saxony_trajectories.csv",index=False)

In [5]:
d=pd.read_csv("saxony_trajectories.csv",header=0)
d

<IPython.core.display.Javascript object>

,traj_id,latitude,longitude,city,state,country
0,758.0,50.8572,13.4729,Bobritzsch-Hilbersdorf,Saxony,Germany
1,421.0,51.1189,14.4454,NaN,Saxony,Germany
2,270.0,50.8357,12.9280,Chemnitz,Saxony,Germany
3,975.0,51.0332,13.7856,Dresden,Saxony,Germany
4,670.0,50.6909,13.0306,NaN,Saxony,Germany
...,...,...,...,...,...,...
1002,100.0,51.0854,13.7372,NaN,Saxony,Germany
1003,216.0,51.0712,13.6445,NaN,Saxony,Germany
1004,518.0,51.0720,13.7909,Dresden,Saxony,Germany
1005,4.0,50.9658,14.7018,NaN,Saxony,Germany


In [6]:
saxony_trajeectory_id=list(d.traj_id.values.astype(int))

## map-mathcing original trajectories

In [ ]:
import os
df_concat1 = pd.DataFrame(columns=["traj_id","latitude","longitude","city"])
directory = os.path.join("/home/dadwal/german_watermarkedData/watermarked_data/")
for root,dirs,files in os.walk(directory):
    for dir1 in dirs:
        if dir1!="watermarking":
            df_2 = pd.read_csv('/home/dadwal/german_watermarkedData/watermarked_data/'+str(dir1)+"/watermarking/watermarkedTraj.csv")
            df_2=df_2[["traj_id","latitude","longitude","time"]]
            if int(df_2.traj_id.values[0]) in saxony_trajeectory_id:
                df=df_2[["time","longitude","latitude",]]
                df.columns=['gps_time', 'longitude', 'latitude']
                
                df['gps_time'] = pd.to_datetime(df['gps_time'])
                df = df.sort_values(by=['gps_time'])

                df_trip_for_meili = df[['longitude', 'latitude', 'gps_time']].copy()
                df_trip_for_meili.columns = ['lon', 'lat', 'time']    
                
                meili_coordinates = df_trip_for_meili.to_json(orient='records')
                meili_head = '{"shape":'
                meili_tail = ""","search_radius": 150, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}"""
                meili_request_body = meili_head + meili_coordinates + meili_tail
                
                # run docker for vallah map matching
                url = "http://localhost:8002/trace_route"
                headers = {'Content-type': 'application/json'}
                data = str(meili_request_body)

                r = requests.post(url, data=data, headers=headers)
               

                if r.status_code == 200:
    
                    response_text = json.loads(r.text)

                    resp = str(response_text['tracepoints'])

                    resp = resp.replace("'waypoint_index': None", "'waypoint_index': '#'")
                    resp = resp.replace("None", "{'matchings_index': '#', 'name': '', 'waypoint_index': '#', 'alternatives_count': 0, 'distance': 0, 'location': [0.0, 0.0]}")

                    resp = resp.replace("'", '"')

                    resp = json.dumps(resp)
                    resp = json.loads(resp)

                    df_response = pd.read_json(resp)
                    df_response = df_response[['name', 'distance', 'location']]

                    df_trip_optimized = pd.merge(df_trip_for_meili, df_response, left_index=True, right_index=True)

                    for i, row in df_trip_optimized.iterrows():
                            df_trip_optimized.at[i, 'longitude'] = df_trip_optimized.at[i,'location'][0]
                            df_trip_optimized.at[i, 'latitude'] = df_trip_optimized.at[i,'location'][1]

                            #saving all the geopoints that couldn't be snapped to road
                            if df_trip_optimized.at[i, 'longitude'] == 0.0:
                                df_trip_optimized.at[i, 'longitude'] = df_trip_optimized.at[i,'lon']
                                df_trip_optimized.at[i, 'latitude'] = df_trip_optimized.at[i,'lat']

                    df_trip_optimized["traj_id"] = df_2.traj_id.values[0]
                    df_trip_optimized = df_trip_optimized.drop(['location', 'lon', 'lat'], 1)
                    display(df_trip_optimized.head())
                    df_trip_optimized.to_csv("original_data_mapped/"+str(df_2.traj_id.values[0])+".csv",index=False)

## map matching watermarked trajectories

In [ ]:
import os
df_concat1 = pd.DataFrame(columns=["traj_id","latitude","longitude","city"])
directory = os.path.join("/home/dadwal/german_watermarkedData/watermarked_data/")
for root,dirs,files in os.walk(directory):
    for dir1 in dirs:
        if dir1!="watermarking":
            df_2 = pd.read_csv('/home/dadwal/german_watermarkedData/watermarked_data/'+str(dir1)+"/watermarking/watermarkedTraj.csv")
            df_2=df_2[["traj_id","watermarked_lat","watermarked_long","time"]]
            if int(df_2.traj_id.values[0]) in saxony_trajeectory_id:
                df=df_2[["time","watermarked_long","watermarked_lat",]]
                df.columns=['gps_time', 'longitude', 'latitude']
                
                df['gps_time'] = pd.to_datetime(df['gps_time'])
                df = df.sort_values(by=['gps_time'])

                df_trip_for_meili = df[['longitude', 'latitude', 'gps_time']].copy()
                df_trip_for_meili.columns = ['lon', 'lat', 'time']
                #df_trip_for_meili.head()
                
                
                meili_coordinates = df_trip_for_meili.to_json(orient='records')
                meili_head = '{"shape":'
                meili_tail = ""","search_radius": 150, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}"""
                meili_request_body = meili_head + meili_coordinates + meili_tail

                url = "http://localhost:8002/trace_route"
                headers = {'Content-type': 'application/json'}
                data = str(meili_request_body)

                r = requests.post(url, data=data, headers=headers)
               

                if r.status_code == 200:
    
                    response_text = json.loads(r.text)

                    resp = str(response_text['tracepoints'])

                    resp = resp.replace("'waypoint_index': None", "'waypoint_index': '#'")
                    resp = resp.replace("None", "{'matchings_index': '#', 'name': '', 'waypoint_index': '#', 'alternatives_count': 0, 'distance': 0, 'location': [0.0, 0.0]}")

                    resp = resp.replace("'", '"')

                    resp = json.dumps(resp)
                    resp = json.loads(resp)

                    df_response = pd.read_json(resp)
                    df_response = df_response[['name', 'distance', 'location']]

                    df_trip_optimized = pd.merge(df_trip_for_meili, df_response, left_index=True, right_index=True)

                    for i, row in df_trip_optimized.iterrows():
                            df_trip_optimized.at[i, 'longitude'] = df_trip_optimized.at[i,'location'][0]
                            df_trip_optimized.at[i, 'latitude'] = df_trip_optimized.at[i,'location'][1]

                            #saving all the geopoints that couldn't be snapped to road
                            if df_trip_optimized.at[i, 'longitude'] == 0.0:
                                df_trip_optimized.at[i, 'longitude'] = df_trip_optimized.at[i,'lon']
                                df_trip_optimized.at[i, 'latitude'] = df_trip_optimized.at[i,'lat']

                    df_trip_optimized["traj_id"] = df_2.traj_id.values[0]
                    df_trip_optimized = df_trip_optimized.drop(['location', 'lon', 'lat'], 1)
                    display(df_trip_optimized.head())
                    df_trip_optimized.to_csv("watermarked_data_mapped/"+str(df_2.traj_id.values[0])+".csv",index=False)
            

In [16]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union

In [ ]:
directory_watermarked = os.path.join("/home/dadwal/gisops_docker/watermarked_data_mapped/")
directory_raw = os.path.join("/home/dadwal/gisops_docker/original_data_mapped/")
sim_list=[]
for root_w,dirs_w,files_w in os.walk(directory_watermarked):
    for root_o,dirs_o,files_o in os.walk(directory_raw):
        for file_w in files_w:
            for file_o in files_o:
                if file_w==file_o:
                    print(file_w,file_o)
                    df_w = pd.read_csv('/home/dadwal/gisops_docker/watermarked_data_mapped/'+file_w)
                    df_o= pd.read_csv('/home/dadwal/gisops_docker/original_data_mapped/'+file_o)
                    df_w=df_w.fillna("same")
                    df_o=df_o.fillna("same")
                    ls_w_streetname=list(df_w.name.values)
                    ls_o_streetname=list(df_o.name.values)
                    print(ls_w_streetname)
                    sim=jaccard_similarity(ls_w_streetname,ls_o_streetname)
                    sim_list.append(sim)

In [22]:
import statistics
statistics.mean(sim_list)

0.9814743776212823